~loading dataset and then printing first row and size of data to understand the structure

~removing unnecessary coloumns, punctuations and stopwords

~making all text lower case

~tokenizng

In [44]:
import json
from collections import Counter
record = [json.loads(line) for line in open('record.json')]  # Loading data from record.json

print("Size of dataset:", len(record))
print(" ")
print("Printing first row before preprocessing:")  #seeing coloumns by printing first line
print(record[0])

remove_col = ['helpful', 'unixReviewTime', 'reviewTime'] #Making a list and including coloumns which are unnecessary for deleting
for data in record:
    for column in remove_col:
        if column in data:
            del data[column]


lowercase_keys = ['reviewerName', 'reviewText', 'summary'] # Making a list and including coloumns which are lower cased
for data in record:
    for key in lowercase_keys:
        if key in data and isinstance(data[key], str):
            data[key] = data[key].lower()


punctuation_list = ['!', '"', '&', "'", '(', ')', ',', '-', '.', ':', ';', '?']  # Removing punctuations from string values by making a list of punctuation 
for data in record:
    for key in lowercase_keys:                   
        if key in data and isinstance(data[key], str):
            without_punc = "".join(char for char in data[key] if char not in punctuation_list)
            data[key] = without_punc

stopwords_list = []                     # Loading stopwords from stopwords.txt
with open('stopwords.txt') as file:
    stopwords_list = [line.strip() for line in file]
for data in record:               # Tokenizing text and removing stopwords
    for key in lowercase_keys:
        if key in data and isinstance(data[key], str):
            words = data[key].split()
            without_stopwords = [word for word in words if word not in stopwords_list]
            data[key] = ' '.join(without_stopwords)
print(" ")            
print("Printing first row after preprocessing:")    
print(record[0])


Size of dataset: 194439
 
Printing first row before preprocessing:
{'reviewerID': 'A30TL5EWN6DFXT', 'asin': '120401325X', 'reviewerName': 'christina', 'helpful': [0, 0], 'reviewText': "They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again", 'overall': 4.0, 'summary': 'Looks Good', 'unixReviewTime': 1400630400, 'reviewTime': '05 21, 2014'}
 
Printing first row after preprocessing:
{'reviewerID': 'A30TL5EWN6DFXT', 'asin': '120401325X', 'reviewerName': 'christina', 'reviewText': 'look good stick good dont like rounded shape always bumping siri kept popping irritating wont buy product like', 'overall': 4.0, 'summary': 'looks good'}


~Distinguishing between good and bad review by setting ratings to positive,negative and neutral

~Printing number of times positive,neagtive and neutral ratin occured

~Extracting positive and negative reviews based on rating then finding most common words in both types of reviews and printing

In [45]:
rating_affect = {  # Declaring sentiments for each rating
    1.0: "negative",
    2.0: "negative",
    3.0: "neutral",
    4.0: "positive",
    5.0: "positive"
}
pos_count = 0  # Initializing counters for positive, negative, and neutral ratings
neg_count = 0
neu_count = 0

for data in record: # Categorizing rating 
    ratings = data['overall']
    sentiment = rating_affect.get(ratings, 'neutral')
    if sentiment == 'positive':
        pos_count += 1
    elif sentiment == 'negative':
        neg_count += 1
    else:
        neu_count += 1

print("Frequency of Positive Ratings:", pos_count) # Printing frequency of positive, negative, and neutral ratings
print("Frequency of Neutral Ratings:", neu_count)
print("Frequency of Negative Ratings:", neg_count)



pos_reviews = [] # Initialize empty lists to store positive and negative reviews
neg_reviews = []

for data in record: # Extracting positive and negative reviews based on ratings
    ratings = data['overall']
    sentiment = rating_affect.get(ratings, 'neutral')
    if sentiment == 'positive':
        pos_reviews.append(data['reviewText'])
    elif sentiment == 'negative':
        neg_reviews.append(data['reviewText'])

pos_words = [word for reviewText in pos_reviews for word in (reviewText).split()]# Tokenizing positive reviews and calculate word frequencies
pos_freq = Counter(pos_words)

neg_words = [word for reviewText in neg_reviews for word in (reviewText).split()]# Tokenizing negative reviews and calculate word frequencies
neg_freq = Counter(neg_words)

pos_common = set([word for word, _ in pos_freq.most_common(8)])# Finding 8 most common words in positive and negative reviews
neg_common = set([word for word, _ in neg_freq.most_common(8)])

remove_neg_common_words = [word for word in neg_words if word not in pos_common]# Removing common words from negative words list
neg_freq_excluded = Counter(remove_neg_common_words)# Calculating word frequencies for modified negative words list

pos_sorted = sorted(pos_freq.items(), key=lambda x: x[1], reverse=True) # Arranging words by frequency in descending order
neg_sorted = sorted(neg_freq_excluded.items(), key=lambda x: x[1], reverse=True)


print("\nMost words mentioned in positive reviews:") # Displaying most mentioned words in positive and negative reviews
for word, frequency in pos_sorted[:10]:
    print(word, "-", frequency)
print(" ")
print("\nMost words mentioned in negative reviews:")
for word, frequency in neg_sorted[:10]:
    print(word, "-", frequency)

Frequency of Positive Ratings: 148657
Frequency of Neutral Ratings: 21439
Frequency of Negative Ratings: 24343

Most words mentioned in positive reviews:
phone - 132953
case - 109061
one - 66769
great - 59145
like - 56125
use - 48063
battery - 46153
good - 46045
screen - 46001
well - 41094
 

Most words mentioned in negative reviews:
would - 8261
get - 7289
screen - 7192
product - 5705
dont - 5252
back - 5210
work - 5138
iphone - 4719
even - 4666
time - 4530


~setting weights for specific words according to positive,negative and neutral

~reviewing reviews and calculating sentiment score for each

~declaring reviews positive,negative and neutral based on sentiment score by setting threshold=0.5

~writing sentiments and its score along with reviews in a .txt file

In [49]:
sentiment_points = { # Declaring weights for specific words under positive,negative and neutral
    "positive": {
        "good": 0.8,
        "excellent": 0.8,
        "amazing": 0.8,
        "awesome": 0.8,
        "perfect": 0.8,
        "fantastic": 0.8,
        "superb": 0.8,
        "great": 0.8,
        "impressive": 0.8,
        "top-notch": 0.8,
        "well":0.8
    },
    "negative": {
        "bad": 0.1,
        "poor": 0.1,
        "disappointing": 0.1,
        "cheap": 0.1,
        "inferior": 0.1,
        "flawed": 0.1,
        "defective": 0.1,
        "subpar": 0.1,
        "dismal": 0.1,
        "unsatisfactory": 0.1,
        "broken":0.1,
        "horrible": 0.1,
        "awful": 0.1,
        "dont":0.1,
        "never":0.1
    },
    "neutral": {
        "okay": 0.5,
        "average": 0.5,
        "satisfactory": 0.5,
        "decent": 0.5,
        "acceptable": 0.5,
        "standard": 0.5,
        "adequate": 0.5,
        "tolerable": 0.5,
        "meh": 0.5
    }
}

def cal_sentiment(reviewText):  # Calculating sentiment score of a review
    sentiment_score = 0
    words = reviewText.split()
    for word in words:
        for sentiment, word_weights in sentiment_points.items():
            sentiment_score += word_weights.get(word, 0)
    return sentiment_score

all_reviews = pos_reviews + neg_reviews


with open("result.txt", "w") as file: # Opening a text file to save the results
    file.write("Result:\n\n")
    file.write("Sentiment analysis for the first 5 reviews:") 
    
    for review in all_reviews[:5]: # Analyze sentiment for the first 5 reviews
        sentiment_score = cal_sentiment(review)
        if sentiment_score > 0.5:
            sentiment = "Positive"
        elif sentiment_score < 0.5:
            sentiment = "Negative"
        else:
            sentiment = "Neutral"
            
        file.write(f"Review: '{review}' - Score: {sentiment_score} - Sentiment: {sentiment}\n") #writing sentiment and score with review

    file.write("\n")  
    file.write("Sentiment analysis for the last 5 reviews:\n") 
    
    for review in all_reviews[-5:]: # Analyze sentiment for the last 5 reviews
        sentiment_score = cal_sentiment(review)
        if sentiment_score > 0.5:
            sentiment = "Positive"
        elif sentiment_score < 0.5:
            sentiment = "Negative"
        else:
            sentiment = "Neutral"
            
        file.write(f"Review: '{review}' - Score: {sentiment_score} - Sentiment: {sentiment}\n")

print("Sentiment analysis Results have been saved to results.txt")



Sentiment analysis Results have been saved to results.txt
